In [2]:
import os
import shutil

In [ ]:
def copy_images_from_subfolders(src_folder, target_folder):
    # Create the target_folder if it doesn't exist
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    # Walk through the source folder and its subdirectories
    for root, _, files in os.walk(src_folder):
        for file in files:
            # Check if the file is an image (you can customize this condition based on file extensions)
            if file.lower().endswith(('.jpg')):
                # Get the full path of the image in the subfolder
                src_image_path = os.path.join(root, file)

                # Copy the image to the target folder
                shutil.copy(src_image_path, target_folder)
                print(f"Copied {src_image_path} to {target_folder}")

# Example usage:
src_folder = "/mnt/disks/data1/captured-images/Incheon8"  # Folder containing subfolders with image files
target_folder = "/mnt/disks/data1/captured-images/Incheon8_modified"  # Target folder to copy all the image files

copy_images_from_subfolders(src_folder, target_folder)

In [138]:
name = 'Incheon8'

def get_jpg_filenames_in_folder(folder_path):
    jpg_filenames = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.jpg'):
            jpg_filenames.append(filename[:-4])  # Remove the last 4 characters (i.e., '.jpg')
    return jpg_filenames

def save_list_to_txt(file_path, data_list):
    with open(file_path, 'w') as file:
        for item in data_list:
            file.write(item + '\n')

folder_path = f'/mnt/disks/data1/captured-images/Processed/raw/{name}_modified'

jpg_filenames = get_jpg_filenames_in_folder(folder_path)

# Specify the absolute path for the txt file in the /mnt/disks/data1/captured-images directory
txt_file_path = f'/mnt/disks/data1/captured-images/Processed/raw/{name}_raw.txt'

save_list_to_txt(txt_file_path, jpg_filenames)

print(f"File names saved to '{txt_file_path}'.")

File names saved to '/mnt/disks/data1/captured-images/Incheon8_raw.txt'.


yolo predict를 위해서 원본에 있는 하위폴더들을 하나의 폴더로 통합

In [155]:
import os
import shutil

# 이미지들이 있는 원본 경로를 지정합니다.
source_path = "/mnt/disks/data1/captured-images/Incheon8/20230419/subfolder_1/subfolder_1"

# 하위 폴더로 분할할 이미지 개수를 지정합니다.
images_per_folder = 1500

# 이미지 파일들의 리스트를 얻어옵니다.
image_files = [file for file in os.listdir(source_path) if file.lower().endswith(".jpg")]

# 이미지 파일들을 1500개씩 하위 폴더로 분할하여 저장합니다.
for i in range(0, len(image_files), images_per_folder):
    subfolder_path = os.path.join(source_path, f"subfolder_{i // images_per_folder + 1}")
    os.makedirs(subfolder_path, exist_ok=True)

    for image_file in image_files[i:i + images_per_folder]:
        source_file_path = os.path.join(source_path, image_file)
        destination_file_path = os.path.join(subfolder_path, image_file)

        shutil.move(source_file_path, destination_file_path)
        print(f"Moved {image_file} to {subfolder_path}")


Moved image_20230419_211051.jpg to /mnt/disks/data1/captured-images/Incheon8/20230419/subfolder_1/subfolder_1
Moved image_20230419_191610.jpg to /mnt/disks/data1/captured-images/Incheon8/20230419/subfolder_1/subfolder_1
Moved image_20230419_185024.jpg to /mnt/disks/data1/captured-images/Incheon8/20230419/subfolder_1/subfolder_1
Moved image_20230419_221834.jpg to /mnt/disks/data1/captured-images/Incheon8/20230419/subfolder_1/subfolder_1
Moved image_20230419_231705.jpg to /mnt/disks/data1/captured-images/Incheon8/20230419/subfolder_1/subfolder_1
Moved image_20230419_174555.jpg to /mnt/disks/data1/captured-images/Incheon8/20230419/subfolder_1/subfolder_1
Moved image_20230419_175727.jpg to /mnt/disks/data1/captured-images/Incheon8/20230419/subfolder_1/subfolder_1
Moved image_20230419_173853.jpg to /mnt/disks/data1/captured-images/Incheon8/20230419/subfolder_1/subfolder_1
Moved image_20230419_175731.jpg to /mnt/disks/data1/captured-images/Incheon8/20230419/subfolder_1/subfolder_1
Moved imag

분할코드

In [3]:
from ultralytics import YOLO
model = YOLO("/mnt/disks/data1/aihub/Training/Modified_Data3/runs/detect/train/weights/best.pt")
results = model.predict(source="/mnt/disks/data1/captured-images/Incheon8/20230421/subfolder_6", batch=16, save_txt = True)



    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

image 1/1500 /mnt/disks/data1/captured-images/Incheon8/20230421/subfolder_6/image_20230421_064252.jpg: 512x640 (no detections), 12.4ms
image 2/1500 /mnt/disks/data1/captured-images/Incheon8/20230421/subfolder_6/image_20230421_064257.jpg: 512x640 (no detections), 10.9ms
image 3/1500 /mnt/disks/data1/captured-images/Incheon8/20230421/subfolder_6/image_20230421_064259.jpg: 512x640 (no detections), 10.7ms
image 4/1500 /mnt/disks/data1/captured-images/Incheon8

현재로썬 DATA3가 가장 최적의 모델, 이로 선박, 등대, 부표 인식되는 이미지들을 1차로 걸러냄

In [4]:
def copy_matching_images(src_folder, txt_folder, target_folder):
    # Get the list of .txt files in the txt_folder
    txt_files = [f for f in os.listdir(txt_folder) if f.endswith('.txt')]

    # Create the target_folder if it doesn't exist
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    for txt_file in txt_files:
        # Extract the file name without the extension from the .txt file
        txt_file_name = os.path.splitext(txt_file)[0]

        # Check if there is a corresponding .jpg image with the same name
        jpg_file_path = os.path.join(src_folder, f"{txt_file_name}.jpg")
        if os.path.exists(jpg_file_path):
            # Copy the .jpg image to the target folder
            target_path = os.path.join(target_folder, f"{txt_file_name}.jpg")
            shutil.copy(jpg_file_path, target_path)
            print(f"Copied {txt_file_name}.jpg to {target_folder}")

# Example usage:
name = "Incheon8"
src_folder = f"/mnt/disks/data1/captured-images/Processed/raw/{name}_modified"  # Source folder containing the .jpg images
txt_folder = "/home/aihub_juhoon/runs/detect/predict4/labels"  # Folder containing the .txt files
target_folder = f"/mnt/disks/data1/captured-images/Processed/{name}_1"  # Target folder to copy the matched .jpg images

copy_matching_images(src_folder, txt_folder, target_folder)

Copied image_20230420_055634.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1
Copied image_20230405_040420.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1
Copied image_20230420_000830.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1
Copied image_20230420_053258.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1
Copied image_20230401_212301.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1
Copied image_20230419_180705.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1
Copied image_20230421_100016.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1
Copied image_20230419_180056.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1
Copied image_20230419_180904.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1
Copied image_20230419_200330.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1
Copied image_20230419_201510.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1
Copied ima

Yolo통해서 save_txt로 저장된 것만 따로 폴더 지정해서 이미지 저장

In [5]:
name = 'Incheon8_Preprocess_1'

def get_jpg_filenames_in_folder(folder_path):
    jpg_filenames = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.jpg'):
            jpg_filenames.append(filename[:-4])  # Remove the last 4 characters (i.e., '.jpg')
    return jpg_filenames

def save_list_to_txt(file_path, data_list):
    with open(file_path, 'w') as file:
        for item in data_list:
            file.write(item + '\n')

folder_path = '/mnt/disks/data1/captured-images/Processed/Incheon8_1'

jpg_filenames = get_jpg_filenames_in_folder(folder_path)

# Specify the absolute path for the txt file in the /mnt/disks/data1/captured-images directory
txt_file_path = f'/mnt/disks/data1/captured-images/Processed/Incheon8_1/{name}.txt'

save_list_to_txt(txt_file_path, jpg_filenames)

print(f"File names saved to '{txt_file_path}'.")

File names saved to '/mnt/disks/data1/captured-images/Processed/Incheon8_1/Incheon8_Preprocess_1.txt'.


위는 이미지들의 제목을 txt리스트로 통합해서 저장하는 코딩, 여러차례 쓰이기에 경로랑 제목 유의해서 재설정

In [6]:
from PIL import Image
import numpy as np

def is_night_background(image_path, threshold=100):
    # Open the image using PIL
    image = Image.open(image_path)
    
    # Convert the image to grayscale
    grayscale_image = image.convert("L")
    
    # Convert the PIL image to a numpy array
    np_image = np.array(grayscale_image)
    
    # Calculate the mean pixel intensity of the image
    mean_intensity = np.mean(np_image)
    
    # If the mean intensity is below the threshold, consider it as a night background image
    return mean_intensity < threshold

def get_jpg_filenames_with_night_background(folder_path):
    jpg_filenames = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith('.jpg'):
            image_path = os.path.join(folder_path, filename)
            if is_night_background(image_path):
                jpg_filenames.append(filename[:-4])  # Remove the last 4 characters (i.e., '.jpg')
    return jpg_filenames

def save_list_to_txt(file_path, data_list):
    with open(file_path, 'w') as file:
        for item in data_list:
            file.write(item + '\n')

folder_path = '/mnt/disks/data1/captured-images/Processed/Incheon8_1'

jpg_filenames_night_background = get_jpg_filenames_with_night_background(folder_path)

# Specify the absolute path for the txt file in the /mnt/disks/data1/captured-images directory
txt_file_path = '/mnt/disks/data1/captured-images/Processed/Incheon8_1/night_list.txt'

save_list_to_txt(txt_file_path, jpg_filenames_night_background)

print(f"Night background image names saved to '{txt_file_path}'.")


Night background image names saved to '/mnt/disks/data1/captured-images/Processed/Incheon8_1/night_list.txt'.


밤 배경의 이미지 리스트 추출

In [30]:
from ultralytics import YOLO
model = YOLO("/home/aihub_juhoon/runs/detect/train2/weights/best.pt")

# Replace this with the appropriate path to the folder containing your images
folder_path = "/mnt/disks/data1/captured-images/Processed/Incheon8_1"

# Get a list of all image filenames in the folder
image_filenames = [filename for filename in os.listdir(folder_path) if filename.lower().endswith(".jpg")]

# Filter out the images that include "20230409" in their filenames
selected_image_filenames = [filename for filename in image_filenames if "20230421_1" in filename]

# Iterate over the selected image filenames and make predictions for each image
for image_filename in selected_image_filenames:
    image_path = os.path.join(folder_path, image_filename)
    
    # Call the model.predict() function for each image
    results = model.predict(source=image_path,save_txt=True,stream=False,batch=16,conf=0.7)



image 1/1 /mnt/disks/data1/captured-images/Processed/Incheon8_1/image_20230421_100920.jpg: 512x640 (no detections), 13.1ms
Speed: 3.5ms preprocess, 13.1ms inference, 1.1ms postprocess per image at shape (1, 3, 512, 640)
Results saved to runs/detect/predict6
0 label saved to runs/detect/predict6/labels

image 1/1 /mnt/disks/data1/captured-images/Processed/Incheon8_1/image_20230421_100531.jpg: 512x640 (no detections), 13.8ms
Speed: 3.4ms preprocess, 13.8ms inference, 1.0ms postprocess per image at shape (1, 3, 512, 640)
Results saved to runs/detect/predict6
0 label saved to runs/detect/predict6/labels

image 1/1 /mnt/disks/data1/captured-images/Processed/Incheon8_1/image_20230421_140105.jpg: 512x640 (no detections), 11.1ms
Speed: 3.0ms preprocess, 11.1ms inference, 0.8ms postprocess per image at shape (1, 3, 512, 640)
Results saved to runs/detect/predict6
0 label saved to runs/detect/predict6/labels

image 1/1 /mnt/disks/data1/captured-images/Processed/Incheon8_1/image_20230421_142215.j

In [31]:
def save_list_to_txt(file_path, data_list):
    with open(file_path, 'w') as file:
        for item in data_list:
            file.write(item + '\n')

# Path to the directory containing the txt files
labels_dir = '/home/aihub_juhoon/runs/detect/predict5/labels'

# Get a list of all txt filenames in the directory
txt_filenames = [filename for filename in os.listdir(labels_dir) if filename.lower().endswith('.txt')]

# Extract the filenames without the .txt extension
filenames_without_extension = [filename[:-4] for filename in txt_filenames]

# Specify the absolute path for the txt file in the /mnt/disks/data1/captured-images/Processed/Incheon5_1 directory
txt_file_path = '/mnt/disks/data1/captured-images/Processed/Incheon8_1/extract_list.txt'

# Save the list of filenames to the txt file
save_list_to_txt(txt_file_path, filenames_without_extension)

print(f"File names saved to '{txt_file_path}'.")


File names saved to '/mnt/disks/data1/captured-images/Processed/Incheon8_1/extract_list.txt'.


In [32]:
def merge_txt_files(file_paths, output_file_path):
    merged_set = set()

    # Read the content of each txt file and add it to the merged_set
    for file_path in file_paths:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            for line in lines:
                merged_set.add(line.strip())

    # Save the merged content to the output file
    with open(output_file_path, 'w') as output_file:
        for item in merged_set:
            output_file.write(item + '\n')

# List of input txt files to merge
txt_files = [
    '/mnt/disks/data1/captured-images/Processed/Incheon8_1/night_list.txt',
    '/mnt/disks/data1/captured-images/Processed/Incheon8_1/extract_list.txt'
]

# Output file path for the merged txt file
output_file_path = '/mnt/disks/data1/captured-images/Processed/Incheon8_1/Incheon8_Preprocess_2.txt'

# Merge the txt files and save the unique content to the output file
merge_txt_files(txt_files, output_file_path)

print(f"Files merged and saved to '{output_file_path}'.")


Files merged and saved to '/mnt/disks/data1/captured-images/Processed/Incheon8_1/Incheon8_Preprocess_2.txt'.


In [33]:
import shutil

def move_images_from_list(txt_file_path, target_folder):
    # Read the list of image filenames from the txt file
    with open(txt_file_path, 'r') as file:
        image_filenames = file.read().splitlines()

    # Add the ".jpg" extension to each filename
    image_filenames_with_extension = [f"{filename}.jpg" for filename in image_filenames]

    # Iterate over the image filenames and move each image to the target folder
    for image_filename in image_filenames_with_extension:
        source_path = os.path.join('/mnt/disks/data1/captured-images/Processed/Incheon8_1', image_filename)
        target_path = os.path.join(target_folder, image_filename)
        shutil.move(source_path, target_path)

# Path to the txt file containing the list of image names (without the ".jpg" extension)
txt_file_path = '/mnt/disks/data1/captured-images/Processed/Incheon8_1/Incheon8_Preprocess_2.txt'

# Target folder where you want to move the images
target_folder = '/mnt/disks/data1/captured-images/Processed/Incheon8_1_unwanted'
if not os.path.exists(target_folder):
        os.makedirs(target_folder)
# Move the images listed in the txt file to the target folder
move_images_from_list(txt_file_path, target_folder)

print("Images moved successfully.")


Images moved successfully.


따로 걸러내어야할 이미지들만 모아둔 것, 허나 눈으로 직접 한 번 보길 추천

In [35]:
def copy_matching_txt_files(txt_folder, image_folder, target_folder):
    # Get a list of all image filenames in the image_folder
    image_filenames = [filename for filename in os.listdir(image_folder) if filename.lower().endswith('.jpg')]

    # Extract the filenames without the .jpg extension
    image_filenames_without_extension = [filename[:-4] for filename in image_filenames]

    # Check if the target_folder exists, if not, create it
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)

    # Copy txt files with matching filenames to the target_folder
    for image_filename in image_filenames_without_extension:
        source_path = os.path.join(txt_folder, image_filename + '.txt')
        target_path = os.path.join(target_folder, image_filename + '.txt')
        if os.path.exists(source_path):
            shutil.copyfile(source_path, target_path)
            print(f"File copied: {source_path} to {target_path}")
        else:
            print(f"File not found: {source_path}. Skipping copying.")

# Path to the directory containing the txt files
txt_folder_path = '/home/aihub_juhoon/runs/detect/predict4/labels'

# Path to the folder containing the images (In this case, it will be the image folder where .txt files have the same name as the images)
image_folder_path = '/mnt/disks/data1/captured-images/Processed/Incheon8_1'

# Target folder where you want to copy the txt files
target_folder_path = '/mnt/disks/data1/captured-images/Processed/Incheon8_1/ship_list'

# Copy the txt files that match the image filenames in the image folder to the target folder
copy_matching_txt_files(txt_folder_path, image_folder_path, target_folder_path)

print("Txt files copied successfully.")

File copied: /home/aihub_juhoon/runs/detect/predict4/labels/image_20230421_095448.txt to /mnt/disks/data1/captured-images/Processed/Incheon8_1/ship_list/image_20230421_095448.txt
File copied: /home/aihub_juhoon/runs/detect/predict4/labels/image_20230421_100920.txt to /mnt/disks/data1/captured-images/Processed/Incheon8_1/ship_list/image_20230421_100920.txt
File copied: /home/aihub_juhoon/runs/detect/predict4/labels/image_20230419_180947.txt to /mnt/disks/data1/captured-images/Processed/Incheon8_1/ship_list/image_20230419_180947.txt
File copied: /home/aihub_juhoon/runs/detect/predict4/labels/image_20230421_100531.txt to /mnt/disks/data1/captured-images/Processed/Incheon8_1/ship_list/image_20230421_100531.txt
File copied: /home/aihub_juhoon/runs/detect/predict4/labels/image_20230421_081959.txt to /mnt/disks/data1/captured-images/Processed/Incheon8_1/ship_list/image_20230421_081959.txt
File copied: /home/aihub_juhoon/runs/detect/predict4/labels/image_20230419_181111.txt to /mnt/disks/data1

In [36]:
def check_conditions(file_content, min_width, min_area):
    lines = file_content.strip().split('\n')

    objects_with_label_0 = False
    objects_with_min_width = False
    objects_with_min_area = False

    for line in lines:
        values = line.strip().split()
        label = int(values[0])
        width = float(values[3])
        area = float(values[3]) * float(values[4])

        if label == 0:
            objects_with_label_0 = True

        if width >= min_width:
            objects_with_min_width = True

        if area >= min_area:
            objects_with_min_area = True

    if objects_with_label_0 and objects_with_min_width and objects_with_min_area:
        return True
    else:
        return False

def collect_label_files(folder_path, min_width, min_area):
    label_files = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            file_path = os.path.join(folder_path, filename)
            with open(file_path, 'r') as file:
                file_content = file.read()
            if check_conditions(file_content, min_width, min_area):
                label_files.append(os.path.splitext(filename)[0])  # 파일 이름에서 '.txt' 확장자를 뺀 이름만 저장
    return label_files

# 주어진 폴더에서 조건을 만족하는 txt 파일들을 찾아 리스트로 정리
folder_path = '/mnt/disks/data1/captured-images/Processed/Incheon8_1/ship_list'

min_width_threshold = 0.08
min_area_threshold = 0.000364
resulting_label_list = collect_label_files(folder_path, min_width_threshold, min_area_threshold)

# 결과 리스트를 파일에 저장
output_file_path = '/mnt/disks/data1/captured-images/Processed/Incheon8_1/resulting_label_list.txt'

with open(output_file_path, 'w') as output_file:
    for file in resulting_label_list:
        output_file.write(file + '\n')

print("조건을 만족하는 라벨링 데이터의 파일 이름이 저장되었습니다.")


조건을 만족하는 라벨링 데이터의 파일 이름이 저장되었습니다.


In [37]:
with open("/mnt/disks/data1/captured-images/Processed/Incheon8_1/resulting_label_list.txt", "r") as file:
    label_list = file.read().splitlines()

# 이미지를 이동할 경로를 지정합니다.
destination_path = "/mnt/disks/data1/captured-images/Processed/Incheon8_1/final"

# 이미지들이 있는 원본 경로를 지정합니다.
source_path = "/mnt/disks/data1/captured-images/Processed/Incheon8_1"

# 이미지 파일들의 리스트를 얻어옵니다.
image_files = os.listdir(source_path)

# 이미지 파일들 중에서 리스트에 포함된 제목과 일치하는 파일들을 찾아서 이동합니다.
for image_file in image_files:
    file_name, file_extension = os.path.splitext(image_file)
    if file_name in label_list:
        source_file_path = os.path.join(source_path, image_file)
        destination_file_path = os.path.join(destination_path, image_file)

        # 폴더가 없으면 생성합니다.
        os.makedirs(destination_path, exist_ok=True)

        # 이미지 파일을 이동합니다.
        shutil.move(source_file_path, destination_file_path)
        print(f"Moved {image_file} to {destination_path}")

Moved image_20230421_081959.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1/final
Moved image_20230331_151107.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1/final
Moved image_20230331_154320.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1/final
Moved image_20230421_100311.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1/final
Moved image_20230422_023509.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1/final
Moved image_20230404_170300.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1/final
Moved image_20230331_143250.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1/final
Moved image_20230404_165055.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1/final
Moved image_20230421_075912.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1/final
Moved image_20230421_091153.jpg to /mnt/disks/data1/captured-images/Processed/Incheon8_1/final
Moved image_20230421_100526.jpg to /mnt/disks/data

: 